In [4]:
import os, sys
import scipy.io as scio 
import sys
import yaml
import shutil
import numpy as np
import pandas as pd

#创建目录，返回路径
def create_path(config_data):
    output_path = config_data['path']+"/"+config_data['save_folders']+"/"+config_data['SequenceName']+config_data['SequencePart']+"/"
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        print("created the path:",output_path)
    if not os.path.exists(output_path+"CAM1"):
            os.makedirs(output_path+"CAM1")
            print("created the path:",output_path+"CAM1")
    if not os.path.exists(output_path+"CAM2"):
            os.makedirs(output_path+"CAM2")
            print("created the path:",output_path+"CAM2")     
    if not os.path.exists(output_path+"label"):
            os.makedirs(output_path+"label")
            print("created the path:",output_path+"label")                 
    if not os.path.exists(output_path+"calib"):
            os.makedirs(output_path+"calib")
            print("created the path:",output_path+"calib")  
    #else:
        #print(output_path,"has been exists")
    return output_path
def txt_rename(config_data,path):
    path = path+"label/"
    imageNameFile = config_data['path']+"/"+config_data['SequenceName']+config_data['SequencePart']+"/"+config_data['view1_sequence']
    imageNamedata = scio.loadmat(imageNameFile)
    time_list=imageNamedata['result'][0][0][0][0][0][0][0]
    
    for index,name in enumerate(time_list):
        old_name=path+str(int(name))+".txt"
        new_name=path+"%06d" % (index)+"_LABEL.txt"
        if os.path.isfile(old_name):
            
            os.rename(old_name, new_name)
            print("rename ",old_name,"to ",new_name)
        else:
            data =""
            with open(new_name, 'wt') as f:
                f.write(data)
            #print("not found ",old_name,"create ",new_name)    

    #os.path.exists()假设你想检查文件“test_data”是否存在，但是当前路径下有个叫“test_data”的文件夹，这样就可能出现误判。
    #if os.path.isfile(path+name):
        
def Image_rename(config_data,output_path):  
    cam1_path = config_data['path']+"/"+config_data['SequenceName']+config_data['SequencePart']+"/"+"KAB_SK_1_undist"
    cam2_path = config_data['path']+"/"+config_data['SequenceName']+config_data['SequencePart']+"/"+"KAB_SK_4_undist"
    #======cam1==========  
    files =os.listdir(cam1_path) #采用listdir来读取所有文件
    files.sort() #排序
    s= []                   #创建一个空列表
    for file_ in files:     #循环读取每个文件名
        if not  os.path.isdir(cam1_path +file_):  #判断该文件是否是一个文件夹
            f_name = str(file_)
            s.append(f_name)  # 把当前文件名返加到列表里

    for index, name in enumerate(s):
        print("from:",cam1_path+"/"+name)
        print("to:",output_path+"CAM1"+"/"+"%06d" % (index)+"_CAM1.bmp")
        shutil.copy(cam1_path+"/"+name,output_path+"CAM1"+"/"+"%06d" % (index)+"_CAM1.bmp")
        
#======cam2==========        
    files =os.listdir(cam2_path) #采用listdir来读取所有文件
    files.sort() #排序
    s= []                   #创建一个空列表
    for file_ in files:     #循环读取每个文件名
        if not  os.path.isdir(cam2_path +file_):  #判断该文件是否是一个文件夹
            f_name = str(file_)
            s.append(f_name)  # 把当前文件名返加到列表里

    for index, name in enumerate(s):        
        print("from:",cam2_path+"/"+name)
        print("to:",output_path+"CAM2"+"/"+"%06d" % (index)+"_CAM2.bmp")
        shutil.copy(cam2_path+"/"+name,output_path+"CAM2"+"/"+"%06d" % (index)+"_CAM2.bmp")     
        
def read_config(config_path='./config/config.yaml'):
    file = open(config_path, 'r', encoding="utf-8")
    #读取文件中的所有数据
    file_data = file.read()                 
    file.close()
    #指定Loader
    config_data = yaml.load(file_data,Loader=yaml.FullLoader)    
    return config_data

def Matfile_convert_pd(config_data):
    
    matdataFile = config_data['path']+"/"+config_data['SequenceName']+config_data['SequencePart']+"/"+config_data['labels']

    data = scio.loadmat(matdataFile)
    col=data['result'][0][0][0][0][0][3][0][0][0][0][0]#data的标签
    columns_name=["timestamp"]
    for index,name in enumerate(col):
        columns_name.append(name[0][0])
        #print(index,name)

    list=[]
    for time_index,timestamp in enumerate(data['result']['BoxfittingLabels_REF'][0][0][0][0][0][0][:]):

        #print (time_index,timestamp)
        for col_index,col in enumerate(data['result']['BoxfittingLabels_REF'][0][0][0][0][3][0][time_index][0][0][1][:]):
            #frame.insert(col,obj,values,axis)
            #df.append(df2, ignore_index=True)  # ignore_index设置为True，index将会忽略df2的index
            #print(col)
            #print ("ttt;",np.insert(col,0,timestamp,0))
            list.append(np.insert(col,0,timestamp,0))


        frame = pd.DataFrame(list,columns=columns_name)
    frame.insert(frame.shape[1],"reserved",0)    
    #print(data['result']['BoxfittingLabels_REF'][0][0][0][0][3][0][index][0][0][1])
    return frame

def pd_to_file(config_data,pd,output_path):
    output_path = output_path+"label/"
    print(output_path)
    
    file_name=frame.drop_duplicates(subset = "timestamp")['timestamp'].values.tolist()
    frame[frame["timestamp"]==file_name[0]]
    
    for index,name in enumerate(file_name):
        #由于文件读写时都有可能产生IOError，一旦出错，后面的f.close()就不会调用。
        #所以，为了保证无论是否出错都能正确地关闭文件，我们可以使用try … finally来实现：
        #但是更好的是使用with as语法，会自动关闭
        with open(output_path+str(int(name))+".txt", 'wt') as f:
            file_name=frame.drop_duplicates(subset = "timestamp")['timestamp'].values.tolist()
            one_file=frame[frame["timestamp"]==file_name[0]]
            data=one_file.to_csv(header=0,index=False,columns=["FEAT_CLASSIFICATION",\
                                                          "reserved","reserved","reserved","reserved","reserved","reserved","reserved",\
                                                          "FEAT_HEIGHT","FEAT_WIDTH","FEAT_LENGTH",\
                                                          "FEAT_X_POS","FEAT_Y_POS","FEAT_Z_POS",\
                                                          "FEAT_ORIENTATION_ANGLE"])
            f.write(data)

In [5]:
config_data=read_config('./config/config.yaml')
output_path=create_path(config_data)
frame=Matfile_convert_pd(config_data)
pd_to_file(config_data,frame,output_path)
Image_rename(config_data,output_path)
txt_rename(config_data,output_path)


KeyboardInterrupt: 

In [40]:

matdataFile = config_data['path']+"/"+config_data['SequenceName']+config_data['SequencePart']+"/"+config_data['view1_sequence']

data = scio.loadmat(matdataFile)

In [7]:

matdataFile = config_data['path']+"/"+config_data['SequenceName']+config_data['SequencePart']+"/"+config_data['labels']

data = scio.loadmat(matdataFile)
data['result']

NameError: name 'config_data' is not defined

In [205]:
#data['result'][0][0][0][0][0][3][0][50][0][0][1]
data['result']['BoxfittingLabels_REF'][0][0][0][0][3][0][50][0][0][1]#第50个时间戳，每个里面有对应的

array([[  4.50670893,  14.64945711,   0.        ,   2.        ,
          4.        ,   1.5       ,   2.68666667,   4.        ,
          3.        ,   3.        ],
       [ 15.58688181,   2.74774433,   0.        ,   2.33666667,
          4.36666667,   1.45      ,   4.43      ,   4.        ,
          4.        ,   3.        ],
       [  6.37636084,  -4.37835555,   0.        ,   2.2       ,
          4.2       ,   1.4       ,   0.95      ,   4.        ,
          7.        ,   3.        ],
       [ -2.29214479,  10.57288041,   0.        ,   3.        ,
          7.        ,   1.5       ,  -0.69      ,   4.        ,
          8.        ,   3.        ],
       [ -4.13963581,   8.48939459,   0.        ,   2.4       ,
          4.7       ,   1.5       ,   5.59      ,   4.        ,
          9.        ,   3.        ],
       [ -7.93303421,  15.694787  ,   0.        ,   2.2       ,
          5.3       ,   1.5       ,  -0.72      ,   4.        ,
         10.        ,   3.        ],
       [ -

In [43]:
len(data['result'][0][0][0][0][0][0][0])#时间戳
data['result'][0][0][0][0][0][0][0]#时间戳，重点

array([1.3847793e+15, 1.3847793e+15, 1.3847793e+15, ..., 1.3847794e+15,
       1.3847794e+15, 1.3847794e+15])

In [44]:
len(data['result'][0][0][0][0][0][1])#标签，无用，最后data里有
data['result'][0][0][0][0][0][1]

array([[array(['FEAT_X_POS'], dtype='<U10')],
       [array(['FEAT_Y_POS'], dtype='<U10')],
       [array(['FEAT_Z_POS'], dtype='<U10')],
       [array(['FEAT_WIDTH'], dtype='<U10')],
       [array(['FEAT_LENGTH'], dtype='<U11')],
       [array(['FEAT_HEIGHT'], dtype='<U11')],
       [array(['FEAT_ORIENTATION_ANGLE'], dtype='<U22')],
       [array(['FEAT_REFERENCEPOINT'], dtype='<U19')],
       [array(['FEAT_ID'], dtype='<U7')],
       [array(['FEAT_CLASSIFICATION'], dtype='<U19')]], dtype=object)

In [228]:
data['result'][0][0][0][0][0][1]#result.BoxfittingLabels_REF.convertMethodName无用

array([[array(['FEAT_X_POS'], dtype='<U10')],
       [array(['FEAT_Y_POS'], dtype='<U10')],
       [array(['FEAT_Z_POS'], dtype='<U10')],
       [array(['FEAT_WIDTH'], dtype='<U10')],
       [array(['FEAT_LENGTH'], dtype='<U11')],
       [array(['FEAT_HEIGHT'], dtype='<U11')],
       [array(['FEAT_ORIENTATION_ANGLE'], dtype='<U22')],
       [array(['FEAT_REFERENCEPOINT'], dtype='<U19')],
       [array(['FEAT_ID'], dtype='<U7')],
       [array(['FEAT_CLASSIFICATION'], dtype='<U19')]], dtype=object)

In [236]:
data['result'][0][0][0][0][0][1][label][0][0]#data

'FEAT_WIDTH'

In [116]:
data['result'][0][0][0][0][0][3][0][0][0][0][1]

array([[  8.49954791,  -7.19323196,   0.        ,   2.1       ,
          4.3       ,   1.5       ,   0.82      ,   4.        ,
          2.        ,   3.        ],
       [-26.01264491,  32.29178496,   0.        ,   2.2       ,
          5.3       ,   1.5       ,  -0.74823529,   4.        ,
         13.        ,   3.        ],
       [ 87.11769851, -51.90121001,   0.        ,   2.3       ,
          4.1       ,   1.5       ,   2.45      ,   4.        ,
         44.        ,   3.        ],
       [ 25.3391613 ,  -3.88750495,   0.        ,   2.7       ,
          4.8       ,   1.5       ,   2.43      ,   4.        ,
         58.        ,   3.        ],
       [ -2.61816222,  -4.59403568,   0.        ,   0.6       ,
          0.8       ,   1.7       ,   1.26      ,   4.        ,
         65.        ,   0.        ]])

In [94]:
col=data['result'][0][0][0][0][0][0][0]#时间戳，重点

In [206]:
col=data['result'][0][0][0][0][0][0][0]#时间戳，重点
columns_name=["timestamp"]
for index,name in enumerate(col):
    #print (index,int(name))
    for k in:
        temp_list=[]
        temp_list.append(k)
        temp_list.insert(0,int(name))
        print(temp_list)

data['result']['BoxfittingLabels_REF'][0][0][0][0][3][0][50][0][0][1]#第50个时间戳，每个里面有对应的
    
    



[1384779301239990, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779301239990, array([-26.01264491,  32.29178496,   0.        ,   2.2       ,
         5.3       ,   1.5       ,  -0.74823529,   4.        ,
        13.        ,   3.        ])]
[1384779301239990, array([ 87.11769851, -51.90121001,   0.        ,   2.3       ,
         4.1       ,   1.5       ,   2.45      ,   4.        ,
        44.        ,   3.        ])]
[1384779301239990, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779301239990, array([-2.61816222, -4.59403568,  0.        ,  0.6       ,  0.8       ,
        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779301320007, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2. 

        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779313720001, array([-2.61816222, -4.59403568,  0.        ,  0.6       ,  0.8       ,
        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779313800018, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779313800018, array([-26.01264491,  32.29178496,   0.        ,   2.2       ,
         5.3       ,   1.5       ,  -0.74823529,   4.        ,
        13.        ,   3.        ])]
[1384779313800018, array([ 87.11769851, -51.90121001,   0.        ,   2.3       ,
         4.1       ,   1.5       ,   2.45      ,   4.        ,
        44.        ,   3.        ])]
[1384779313800018, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779313800018, array([-2.61816222, -4.59403568,  0.        ,  0

        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779325720001, array([-26.01264491,  32.29178496,   0.        ,   2.2       ,
         5.3       ,   1.5       ,  -0.74823529,   4.        ,
        13.        ,   3.        ])]
[1384779325720001, array([ 87.11769851, -51.90121001,   0.        ,   2.3       ,
         4.1       ,   1.5       ,   2.45      ,   4.        ,
        44.        ,   3.        ])]
[1384779325720001, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779325720001, array([-2.61816222, -4.59403568,  0.        ,  0.6       ,  0.8       ,
        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779325800018, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779325800018, array([-26.01264491,  32.29178496,   0.        ,

        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779339079986, array([-2.61816222, -4.59403568,  0.        ,  0.6       ,  0.8       ,
        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779339160003, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779339160003, array([-26.01264491,  32.29178496,   0.        ,   2.2       ,
         5.3       ,   1.5       ,  -0.74823529,   4.        ,
        13.        ,   3.        ])]
[1384779339160003, array([ 87.11769851, -51.90121001,   0.        ,   2.3       ,
         4.1       ,   1.5       ,   2.45      ,   4.        ,
        44.        ,   3.        ])]
[1384779339160003, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779339160003, array([-2.61816222, -4.59403568,  0.        ,  0

        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779349239990, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779349239990, array([-26.01264491,  32.29178496,   0.        ,   2.2       ,
         5.3       ,   1.5       ,  -0.74823529,   4.        ,
        13.        ,   3.        ])]
[1384779349239990, array([ 87.11769851, -51.90121001,   0.        ,   2.3       ,
         4.1       ,   1.5       ,   2.45      ,   4.        ,
        44.        ,   3.        ])]
[1384779349239990, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779349239990, array([-2.61816222, -4.59403568,  0.        ,  0.6       ,  0.8       ,
        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779349320007, array([ 8.49954791, -7.19323196,  0.        ,  2

        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779361880004, array([-26.01264491,  32.29178496,   0.        ,   2.2       ,
         5.3       ,   1.5       ,  -0.74823529,   4.        ,
        13.        ,   3.        ])]
[1384779361880004, array([ 87.11769851, -51.90121001,   0.        ,   2.3       ,
         4.1       ,   1.5       ,   2.45      ,   4.        ,
        44.        ,   3.        ])]
[1384779361880004, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779361880004, array([-2.61816222, -4.59403568,  0.        ,  0.6       ,  0.8       ,
        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779361960021, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779361960021, array([-26.01264491,  32.29178496,   0.        ,

        44.        ,   3.        ])]
[1384779373880004, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779373880004, array([-2.61816222, -4.59403568,  0.        ,  0.6       ,  0.8       ,
        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779373960021, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779373960021, array([-26.01264491,  32.29178496,   0.        ,   2.2       ,
         5.3       ,   1.5       ,  -0.74823529,   4.        ,
        13.        ,   3.        ])]
[1384779373960021, array([ 87.11769851, -51.90121001,   0.        ,   2.3       ,
         4.1       ,   1.5       ,   2.45      ,   4.        ,
        44.        ,   3.        ])]
[1384779373960021, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5  

        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779384839996, array([-26.01264491,  32.29178496,   0.        ,   2.2       ,
         5.3       ,   1.5       ,  -0.74823529,   4.        ,
        13.        ,   3.        ])]
[1384779384839996, array([ 87.11769851, -51.90121001,   0.        ,   2.3       ,
         4.1       ,   1.5       ,   2.45      ,   4.        ,
        44.        ,   3.        ])]
[1384779384839996, array([25.3391613 , -3.88750495,  0.        ,  2.7       ,  4.8       ,
        1.5       ,  2.43      ,  4.        , 58.        ,  3.        ])]
[1384779384839996, array([-2.61816222, -4.59403568,  0.        ,  0.6       ,  0.8       ,
        1.7       ,  1.26      ,  4.        , 65.        ,  0.        ])]
[1384779384920013, array([ 8.49954791, -7.19323196,  0.        ,  2.1       ,  4.3       ,
        1.5       ,  0.82      ,  4.        ,  2.        ,  3.        ])]
[1384779384920013, array([-26.01264491,  32.29178496,   0.        ,

In [70]:
frame

,timestamp,FEAT_X_POS,FEAT_Y_POS,FEAT_Z_POS,FEAT_WIDTH,FEAT_LENGTH,FEAT_HEIGHT,FEAT_ORIENTATION_ANGLE,FEAT_REFERENCEPOINT,FEAT_ID,FEAT_CLASSIFICATION


In [48]:
data['result'][0][0][0][0][0][3][0][0][0][0][1]#data的feature，重点

array([[  8.49954791,  -7.19323196,   0.        ,   2.1       ,
          4.3       ,   1.5       ,   0.82      ,   4.        ,
          2.        ,   3.        ],
       [-26.01264491,  32.29178496,   0.        ,   2.2       ,
          5.3       ,   1.5       ,  -0.74823529,   4.        ,
         13.        ,   3.        ],
       [ 87.11769851, -51.90121001,   0.        ,   2.3       ,
          4.1       ,   1.5       ,   2.45      ,   4.        ,
         44.        ,   3.        ],
       [ 25.3391613 ,  -3.88750495,   0.        ,   2.7       ,
          4.8       ,   1.5       ,   2.43      ,   4.        ,
         58.        ,   3.        ],
       [ -2.61816222,  -4.59403568,   0.        ,   0.6       ,
          0.8       ,   1.7       ,   1.26      ,   4.        ,
         65.        ,   0.        ]])

In [225]:
print(data['result']['BoxfittingLabels_REF'][0][0][0][0][3][0][0][0][0][1])

[[  8.49954791  -7.19323196   0.           2.1          4.3
    1.5          0.82         4.           2.           3.        ]
 [-26.01264491  32.29178496   0.           2.2          5.3
    1.5         -0.74823529   4.          13.           3.        ]
 [ 87.11769851 -51.90121001   0.           2.3          4.1
    1.5          2.45         4.          44.           3.        ]
 [ 25.3391613   -3.88750495   0.           2.7          4.8
    1.5          2.43         4.          58.           3.        ]
 [ -2.61816222  -4.59403568   0.           0.6          0.8
    1.7          1.26         4.          65.           0.        ]]


In [26]:
#data['result'][0][0][0][0][0][3][0][0][0][0][2]#data的availableStatus，无用
#data['result'][0][0][0][0][0][3][0][0][0][0][3]#data的availableCovFeatures无用

rename  /temp_dataset/convert_output/Sequence1a/calib/1384779301399993.txt to  /temp_dataset/convert_output/Sequence1a/calib/000001_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779301480010.txt to  /temp_dataset/convert_output/Sequence1a/calib/000003_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779301559997.txt to  /temp_dataset/convert_output/Sequence1a/calib/000005_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779301640014.txt to  /temp_dataset/convert_output/Sequence1a/calib/000007_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779301720001.txt to  /temp_dataset/convert_output/Sequence1a/calib/000009_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779301800018.txt to  /temp_dataset/convert_output/Sequence1a/calib/000011_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779301880004.txt to  /temp_dataset/convert_output/Sequence1a/calib/000013_LABEL.txt
rename

rename  /temp_dataset/convert_output/Sequence1a/calib/1384779317720001.txt to  /temp_dataset/convert_output/Sequence1a/calib/000409_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779317800018.txt to  /temp_dataset/convert_output/Sequence1a/calib/000411_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779317880004.txt to  /temp_dataset/convert_output/Sequence1a/calib/000413_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779317960021.txt to  /temp_dataset/convert_output/Sequence1a/calib/000415_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779318039978.txt to  /temp_dataset/convert_output/Sequence1a/calib/000417_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779318119995.txt to  /temp_dataset/convert_output/Sequence1a/calib/000419_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779318199981.txt to  /temp_dataset/convert_output/Sequence1a/calib/000421_LABEL.txt
rename

rename  /temp_dataset/convert_output/Sequence1a/calib/1384779343960021.txt to  /temp_dataset/convert_output/Sequence1a/calib/001065_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779344039978.txt to  /temp_dataset/convert_output/Sequence1a/calib/001067_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779344119995.txt to  /temp_dataset/convert_output/Sequence1a/calib/001069_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779344199981.txt to  /temp_dataset/convert_output/Sequence1a/calib/001071_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779344279998.txt to  /temp_dataset/convert_output/Sequence1a/calib/001073_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779344359985.txt to  /temp_dataset/convert_output/Sequence1a/calib/001075_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779344440002.txt to  /temp_dataset/convert_output/Sequence1a/calib/001077_LABEL.txt
rename

rename  /temp_dataset/convert_output/Sequence1a/calib/1384779366760009.txt to  /temp_dataset/convert_output/Sequence1a/calib/001635_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779366839996.txt to  /temp_dataset/convert_output/Sequence1a/calib/001637_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779366920013.txt to  /temp_dataset/convert_output/Sequence1a/calib/001639_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779367000000.txt to  /temp_dataset/convert_output/Sequence1a/calib/001641_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779367079986.txt to  /temp_dataset/convert_output/Sequence1a/calib/001643_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779367160003.txt to  /temp_dataset/convert_output/Sequence1a/calib/001645_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779367239990.txt to  /temp_dataset/convert_output/Sequence1a/calib/001647_LABEL.txt
rename

rename  /temp_dataset/convert_output/Sequence1a/calib/1384779386920013.txt to  /temp_dataset/convert_output/Sequence1a/calib/002139_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779387000000.txt to  /temp_dataset/convert_output/Sequence1a/calib/002141_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779387079986.txt to  /temp_dataset/convert_output/Sequence1a/calib/002143_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779387160003.txt to  /temp_dataset/convert_output/Sequence1a/calib/002145_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779387239990.txt to  /temp_dataset/convert_output/Sequence1a/calib/002147_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779387320007.txt to  /temp_dataset/convert_output/Sequence1a/calib/002149_LABEL.txt
rename  /temp_dataset/convert_output/Sequence1a/calib/1384779387399993.txt to  /temp_dataset/convert_output/Sequence1a/calib/002151_LABEL.txt
rename

In [162]:
list=[]
for time_index,timestamp in enumerate(data['result']['BoxfittingLabels_REF'][0][0][0][0][0][0][:]):
    
    #print (time_index,timestamp)
    for col_index,col in enumerate(data['result']['BoxfittingLabels_REF'][0][0][0][0][3][0][time_index][0][0][1][:]):
        #frame.insert(col,obj,values,axis)
        #df.append(df2, ignore_index=True)  # ignore_index设置为True，index将会忽略df2的index
        #print(col)
        #print ("ttt;",np.insert(col,0,timestamp,0))
        list.append(np.insert(col,0,timestamp,0))
        
    

NameError: name 'data' is not defined

In [387]:
data['result']['BoxfittingLabels_REF'][0][0][0][0][3][0][0][0][0][index][0][0][0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed